# Classification

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.feature_selection import chi2
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from collections import Counter
import numpy as np
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support,accuracy_score
from sklearn import tree
import xgboost as xgb

import warnings
warnings.simplefilter('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import files
classification_file = files.upload()

Saving classification_file.xlsx to classification_file.xlsx


In [ ]:
classification_file=pd.read_excel(r'classification_file.xlsx')

In [ ]:
# 78 around
classification_train = classification_file[[ 'count_at', 'anger', 'anticipation', 'disgust', 'fear',
       'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14',
       '15', '16', '17', '18', '19', 'pros_title_length_y','count_mentions', 'count_video', 'count_words',
       'count_uppercased', 'count_lowercased', 'count_punctuation',
       'total_shares','LABEL',
       'number_of_authors', 'polarity','Average_eigen_vector','Average_degree_centrality','Average_closeness']]
y =  classification_train.LABEL
X =  classification_train.drop('LABEL',axis=1)
scaler=StandardScaler()
scaled_X = scaler.fit_transform(X)


In [ ]:
classification_train = classification_file[['pros_title_length_y','count_mentions', 'count_video', 'count_words',
       'count_uppercased', 'count_lowercased', 'count_punctuation','count_words_title',
       'total_shares','LABEL', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       'number_of_authors', 'polarity','Average_eigen_vector','Average_degree_centrality','Average_betweeness','Average_closeness']]
y =  classification_train.LABEL
X =  classification_train.drop('LABEL',axis=1)
scaler=StandardScaler()
scaled_X = scaler.fit_transform(X)

In [ ]:
def classify(X,y,classifier):
    kf = StratifiedKFold(n_splits=10,shuffle=True,random_state=100)
    acc_score=[]
    prec_score =[]
    f_score =[]
    reca_score =[]
    i=1
    for train_index,test_index in kf.split(X,y):
        #print('{} of KFold {}'.format(i,kf.n_splits))
        xtr,xvl = X[train_index],X[test_index]
        ytr,yvl = y[train_index],y[test_index]

        #model
        clfr = classifier
        clfr.fit(xtr,ytr)
        pred2= clfr.predict(xvl)
        Accuracy= accuracy_score(yvl, pred2)
        #confusion matrix',confusion_matrix(yvl, pred))
        recall = metrics.recall_score(yvl, pred2)
        precision = metrics.precision_score(yvl, pred2)
        fscore=metrics.f1_score(yvl, pred2)
        #print(metrics.confusion_matrix(yvl,pred2))

        #print('confusion matrix',confusion_matrix(yvl, pred2))
        #print('precision_recall_fscore_support',precision_recall_fscore_support(yvl, pred2))
        acc_score.append(Accuracy)
        prec_score.append(precision)
        reca_score.append(recall)
        f_score.append(fscore)
        i+=1
    e = (sum(acc_score)/len(acc_score))
    f= (sum(prec_score)/len(prec_score))
    g=(sum(reca_score)/len(reca_score))
    h=(sum(f_score)/len(f_score))
    print(classifier,':')
    print('\033[1m'+'Average of all Precision :' +'\033[0m',f)
    print('\033[1m'+'Average of all Accuracy :' +'\033[0m',e)
    print('\033[1m'+'Average of all Recall :' +'\033[0m',g)
    print('\033[1m'+'Average of all F1-score :' +'\033[0m',h)
    return e,f,g,h



In [ ]:
def call_classify(X,y):    #our dataset is not imbalanced
    accuracy=[]
    avg_precision=[]
    recall=[]
    f_score=[]
    classifiers = [RandomForestClassifier(n_estimators=200,n_jobs=-1,random_state=100),
                   LogisticRegression(random_state=100),SVC(kernel='linear',random_state=100,probability=True),
                   ExtraTreesClassifier(n_estimators=100, random_state=100),GaussianNB(),
                   xgb.XGBClassifier(objective="binary:logistic", random_state=100)]
    for classifier in (classifiers):
            e,f,g,h = classify(X,y,classifier)
            accuracy.append(e*100)
            avg_precision.append(f*100)
            recall.append(g*100)
            f_score.append(h*100)
    return accuracy,f_score,avg_precision,recall
classif = ['Random Forest','Logistic_regression','SVM','Extratrees classifier','GaussianNB','XGBoost']

In [ ]:
e1,f1,g1,h1=call_classify(scaled_X,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=-1, oob_score=False, random_state=100, verbose=0,
                       warm_start=False) :
Average of all Precision : 0.8170887445887447
Average of all Accuracy : 0.803156146179402
Average of all Recall : 0.7913419913419915
Average of all F1-score : 0.8005217408166926
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=100, solver='lbfgs', tol=0.0001,

In [ ]:
result = pd.DataFrame(
    {'Classifier': classif,
     'Accuracy': e1,
     'Precision': f1,
     'Recall': g1,
     'F1-score': h1,
    })

result

,Classifier,Accuracy,Precision,Recall,F1-score
0,Random Forest,80.315615,80.052174,81.708874,79.134199
1,Logistic_regression,77.729790,78.152053,76.957039,79.610390
2,SVM,78.205980,78.340536,78.455761,78.658009
3,Extratrees classifier,77.696567,77.427754,79.033387,76.298701
4,GaussianNB,66.810631,67.642208,66.836945,69.155844
5,XGBoost,78.909192,78.371961,80.765991,76.298701
